## Mounting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Importing Dependencies

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers
import os
import numpy as np
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

## Gathering Data

In [3]:
(train_data, test_data), ds_info = tfds.load(name="food101", split=["train", "validation"], shuffle_files=False, as_supervised=True, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/75750 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incompleteJ2XR08/food101-train.tfrecord*...:   0%|          …

Generating validation examples...:   0%|          | 0/25250 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incompleteJ2XR08/food101-validation.tfrecord*...:   0%|     …

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


## Accessing & Visualizing Data

In [4]:
# Features of Food101 TFDS
ds_info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=101),
})

In [6]:
# Get class names
class_names = ds_info.features["label"].names
sample = train_data.take(1)

# Output info about our training sample
for image, label in sample:
  print(f"""
  Image shape: {image.shape}
  Image dtype: {image.dtype}
  Target class from Food101 (tensor form): {label}
  Class name (str form): {class_names[label.numpy()]}
  """)


  Image shape: (512, 512, 3)
  Image dtype: <dtype: 'uint8'>
  Target class from Food101 (tensor form): 56
  Class name (str form): huevos_rancheros
  


## Data Preprocessing

In [7]:
#Preprocess Image to 224,224,3
def preprocess_img(image, label, img_shape=224):
  image = tf.image.resize(image, [img_shape, img_shape])
  return tf.cast(image, tf.float32), label

In [8]:
preprocessed_img = preprocess_img(image, label)[0]
print(f"Shape: {preprocessed_img.shape},\nDatatype: {preprocessed_img.dtype}")

Shape: (224, 224, 3),
Datatype: <dtype: 'float32'>


In [9]:
# Preprocess training data, Shuffle
train_data = train_data.map(preprocess_img)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32)

# Preprocess test data
test_data = test_data.map(preprocess_img)
test_data = test_data.batch(32)

## Build feature extraction model


In [10]:
# Mixed Precision Training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy="mixed_float16")
mixed_precision.global_policy()

<Policy "mixed_float16">

In [11]:
# Create Base Model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Create Functional model
inputs = layers.Input(shape=input_shape, dtype=tf.float16)
x = base_model(inputs, training=False)

# Adding our layers
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(.3)(x)
x = layers.Dense(len(class_names))(x)

# Final Output Layers
outputs = layers.Activation("softmax", dtype=tf.float32)(x)

# Compile the model
model = tf.keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])
model.summary()

16705208/16705208 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 101)               129381    
                                                              

In [12]:
history = model.fit(train_data, epochs=10, steps_per_epoch=len(train_data), validation_data=test_data, validation_steps=int(0.15 * len(test_data)))

Epoch 1/10
2368/2368 [==============================] - 219s 84ms/step - loss: 1.8500 - accuracy: 0.5482 - val_loss: 1.1650 - val_accuracy: 0.6907
Epoch 2/10
2368/2368 [==============================] - 201s 83ms/step - loss: 1.3611 - accuracy: 0.6477 - val_loss: 1.0725 - val_accuracy: 0.7127
Epoch 3/10
2368/2368 [==============================] - 193s 80ms/step - loss: 1.2547 - accuracy: 0.6703 - val_loss: 1.0330 - val_accuracy: 0.7188
Epoch 4/10
2368/2368 [==============================] - 191s 79ms/step - loss: 1.2035 - accuracy: 0.6818 - val_loss: 1.0099 - val_accuracy: 0.7217
Epoch 5/10
2368/2368 [==============================] - 189s 78ms/step - loss: 1.1655 - accuracy: 0.6896 - val_loss: 0.9958 - val_accuracy: 0.7307
Epoch 6/10
2368/2368 [==============================] - 191s 78ms/step - loss: 1.1362 - accuracy: 0.6957 - val_loss: 0.9931 - val_accuracy: 0.7262
Epoch 7/10
2368/2368 [==============================] - 190s 78ms/step - loss: 1.1244 - accuracy: 0.6983 - val_loss: 0

In [13]:
# Evaluate model (unsaved version) on whole test dataset
results = model.evaluate(test_data)
results

790/790 [==============================] - 55s 70ms/step - loss: 0.9931 - accuracy: 0.7301


[0.9931296706199646, 0.7301385998725891]

In [14]:
# Create save path to drive
save_dir = "drive/MyDrive/food_app/Model/"
os.makedirs(save_dir)

# Save model
model.save(save_dir)
model.save("/content/drive/MyDrive/food_app/FinalModel.hdf5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Predicting User Input

In [15]:
def load_and_prep_image(filename, img_shape=224):
  img = tf.io.read_file(filename)
  img = tf.image.decode_jpeg(img)
  img = tf.image.resize(img, [img_shape, img_shape])
  return img

def pred_custom(img):
  img = load_and_prep_image(img)
  pred_prob = model.predict(tf.expand_dims(img, axis=0))
  pred_class_index = pred_prob.argmax()
  pred_accuracy = pred_prob[0, pred_class_index]
  print(f"Image: {img}, Predicted Class: {pred_class_index}, Accuracy: {pred_accuracy:.2%}")

In [16]:
pred_custom("/content/paela.jpg")

1/1 [==============================] - 2s 2s/step
Image: [[[ 17.646683   27.468111    4.48852  ]
  [ 20.25       30.07143     6.952806 ]
  [ 21.250002   31.07143     6.8737254]
  ...
  [206.32402   210.32402   219.32402  ]
  [217.96394   221.96394   230.96394  ]
  [215.82133   219.82133   228.82133  ]]

 [[ 20.035715   26.595663   20.832909 ]
  [ 10.83801    16.804848   11.196428 ]
  [ 25.978315   32.920918   25.67857  ]
  ...
  [194.14682   198.14682   207.14682  ]
  [208.65623   212.65623   221.65623  ]
  [219.39423   223.39423   232.39423  ]]

 [[ 41.978313   49.890305   39.154335 ]
  [ 37.095673   45.038273   34.210464 ]
  [ 29.678574   37.667095   26.701532 ]
  ...
  [207.64296   211.64296   220.64296  ]
  [204.88466   208.88466   217.88466  ]
  [198.85051   202.85051   211.85051  ]]

 ...

 [[131.86247   136.86247   165.86247  ]
  [152.55847   157.55847   186.55847  ]
  [139.63316   144.63316   173.63316  ]
  ...
  [173.20193   176.20193   195.20193  ]
  [169.02159   172.02159   